In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# importo modulos de pytorch

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader

import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

### Descargo archivos

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Acceder a una carpeta específica
import os

# Ejemplo de ruta a una carpeta en tu Google Drive
ruta_tini = '/content/drive/MyDrive/proyectos/NLP/TINI'
ruta_spinetta = '/content/drive/MyDrive/proyectos/NLP/Spinetta'
ruta_callejero = '/content/drive/MyDrive/proyectos/NLP/Callejero'

artistas = ['tini', 'spinetta', 'callejero']

tini = []
archivos = os.listdir(ruta_tini)
for archivo in archivos:
    ruta_archivo = os.path.join(ruta_tini, archivo)
    with open(ruta_archivo, 'r') as archivo_txt:
        contenido = archivo_txt.read()
        tini.append(contenido)

spinetta = []
archivos = os.listdir(ruta_spinetta)
for archivo in archivos:
    ruta_archivo = os.path.join(ruta_spinetta, archivo)
    with open(ruta_archivo, 'r') as archivo_txt:
        contenido = archivo_txt.read()
        spinetta.append(contenido)

callejero = []
archivos = os.listdir(ruta_callejero)
for archivo in archivos:
    ruta_archivo = os.path.join(ruta_callejero, archivo)
    with open(ruta_archivo, 'r') as archivo_txt:
        contenido = archivo_txt.read()
        callejero.append(contenido)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
callejero[1]

"Hagan caso\nPa' las rocha', beso\nY pa' los giles, rafagazo' (prra)\nDJ Alex\nEstamo' lo' pibe' al ritmo\nEstá llena la jarra de vino\nBife pa' lo' gile' y pa’ to' sus amigos\nHagan la que quieran, que lo recibimos acá, y\nEstamo' lo' pibe' al ritmo\nEstá llena la jarra de vino\nBife pa' lo' gile' y pa’ to' sus amigos\nHagan la que quieran, que lo recibimos acá, y\nMe muestra en la cara que tiene efectivo\nQuerían bondi, llegué en colectivo\nSuben y bajan cuando yo lo digo\nEsto no es nada, es un correctivo\nYo no me regalo, no compro ni fío\nMenos con ustedes, que son hijos mío'\nLo' miro de lejos de ustede' y me río\nSigan loreando, yo no me persigo\nArrancá con fuerza, gato, y dispará\nQue vos no sos chorro, tampoco te da\nEstamos atentos si doblan pa' acá\nNosotros al ritmo y se quieren matar\nTe hacen pagar y después tocar\nYo voy a cantar, todos a saltar\nYo cuento mi vida, vos sos un sapo má'\nPa' lo' pibe' bueno', pronta libertad\nLlegué de viaje, estaba en Canadá\nTe traje un

### Limpieza de datos y creación del dataset

In [6]:
# Idea: si dejamos los \n capaz aprende cuando poner \n. Los dejo pero separo las letras

for artista in artistas:
  canciones = eval(artista)
  for i in range(len(canciones)):
    # Cambio los "X\nX" por "X \n X"
    canciones[i] = canciones[i].replace("\n", " \n ")

In [ ]:
# Prueba
callejero[1]

"Hagan caso \n Pa' las rocha', beso \n Y pa' los giles, rafagazo' (prra) \n DJ Alex \n Estamo' lo' pibe' al ritmo \n Está llena la jarra de vino \n Bife pa' lo' gile' y pa’ to' sus amigos \n Hagan la que quieran, que lo recibimos acá, y \n Estamo' lo' pibe' al ritmo \n Está llena la jarra de vino \n Bife pa' lo' gile' y pa’ to' sus amigos \n Hagan la que quieran, que lo recibimos acá, y \n Me muestra en la cara que tiene efectivo \n Querían bondi, llegué en colectivo \n Suben y bajan cuando yo lo digo \n Esto no es nada, es un correctivo \n Yo no me regalo, no compro ni fío \n Menos con ustedes, que son hijos mío' \n Lo' miro de lejos de ustede' y me río \n Sigan loreando, yo no me persigo \n Arrancá con fuerza, gato, y dispará \n Que vos no sos chorro, tampoco te da \n Estamos atentos si doblan pa' acá \n Nosotros al ritmo y se quieren matar \n Te hacen pagar y después tocar \n Yo voy a cantar, todos a saltar \n Yo cuento mi vida, vos sos un sapo má' \n Pa' lo' pibe' bueno', pronta li

Además, tokenizo las palabras y elimino las stop words para poder entrenar la red con mejor información.

In [ ]:
artistas

['tini', 'spinetta', 'callejero']

In [7]:
# Armo diccionario dataset donde voy a tener las canciones clasificadas
# para el modelo
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.tokenize import sent_tokenize

dataset = {'tini':[],'callejero':[],'spinetta':[]}


def tokenize(dataset, artistas):

    # Stop words
    stop_words = set(stopwords.words('spanish'))

    # Armo corpus de todas las palabras
    corpus = []

    for artista in artistas:
      canciones = eval(artista)
      for cancion in canciones:
        # Tokenizo la cancion
        word_list = []

        for word in cancion.lower().split():
            if word not in stop_words and word != '':
                # Si no esta, la agrego al corpus
                if word not in corpus:
                  corpus.append(word)
                word_list.append(word)

        # Agrego la cancion etiquetada como el artista
        dataset[artista].append(word_list)

    # Armo el onehot enconding

    # Cuento la cantidad de palabras
    count = Counter(corpus)

    # Creo diccionario onehot
    onehot_dict = {w:i+1 for i,w in enumerate(count)}

    print("Se registraron",len(onehot_dict),"palabras")
    for palabra in onehot_dict.keys():
      # Creo vector de 0s
      a = onehot_dict[palabra]
      onehot_dict[palabra] = [0] * len(onehot_dict)
      onehot_dict[palabra][a-1] = 1

    return dataset, onehot_dict

dataset, onehot_dict = tokenize(dataset, artistas)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Se registraron 3680 palabras


In [8]:
# Armo diccionario dataset donde voy a tener las canciones clasificadas
# para el modelo
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
from collections import Counter
from nltk.tokenize import sent_tokenize

dataset = {'tini':[],'callejero':[],'spinetta':[]}


def tokenize(dataset, artistas):

    # Stop words
    stop_words = set(stopwords.words('spanish'))

    # Armo corpus de todas las palabras
    corpus = []

    for artista in artistas:
      canciones = eval(artista)
      for cancion in canciones:
        # Tokenizo la cancion
        word_list = []
        for word in cancion.lower().split():
            if word not in stop_words and word != '':
                # Si no esta, la agrego al corpus
                if word not in corpus:
                  corpus.append(word)
                # Agrego la cancion etiquetada como artista
                word_list.append(word)

        dataset[artista].append(word_list)

    print("El tama;o del corpus es de",len(corpus),"palabras")

    return dataset, corpus

dataset, corpus = tokenize(dataset, artistas)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


El tama;o del corpus es de 3680 palabras


In [9]:
# Lo convierto en un dataframe de pandas

data = {'cancion':[],'artista':[]}

for artista in artistas:
  canciones = dataset[artista]
  for cancion in canciones:
    data['cancion'].append(' '.join(cancion))
    data['artista'].append(artista)

df = pd.DataFrame(data)
df.head()

,cancion,artista
0,si vas volveré enamorar volveré desarmar vez s...,tini
1,(this is the remix) viendo hace tiempo 'toy es...,tini
2,"acércate, acércate sé brillo ojos malo pasao',...",tini
3,amor dejaste tirado bar entonces pásame botell...,tini
4,"(no, boluda, mirá, hiciste llorar, puta madre)...",tini


Separamos en train y test

In [10]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

X_train = train_df['cancion']
y_train = train_df['artista']

X_test = test_df['cancion']
y_test = test_df['artista']

y_train, categorias = pd.factorize(y_train)
y_test, categorias = pd.factorize(y_test)

### Modelo Clasificador

In [11]:
class Clasificador(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_classes):

        super(Clasificador, self).__init__()

        # Capa de embeddings
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # Capa LSTM
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)

        # Capa totalmente conectada
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):

        # Obtener las embeddings de las palabras
        embedded = self.embedding(x)

        # Pasar por la LSTM
        lstm_out, (hn, cn) = self.lstm(embedded)

        # Tomamos el último estado oculto
        final_hidden_state = hn[-1]

        # Pasamos a través de la capa totalmente conectada
        out = self.fc(final_hidden_state)

        return out

Parámetros del modelo:

    vocab_size: Tamaño del vocabulario.
    embedding_dim: Dimensión de los vectores de embeddings.
    hidden_dim: Número de unidades en la capa LSTM.
    num_classes: Número de clases (en este caso 3: Artista A, B, C).

In [12]:
# Paso el dataset a Pytorch

from torch.utils.data import Dataset

class SongDataset(Dataset):
    def __init__(self, texts, labels, maxlength):
        self.texts = texts  # Lista de listas de tokens (canciones)
        self.labels = labels  # Las etiquetas de clase
        self.max_len = maxlength

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        # Lista de listas de onehot_encoding
        cancion = self.texts[idx]

        # Pongo 3680 a mano porque es el tama;o del onehotencoding
        if len(cancion) < self.max_len:
          cancion = cancion + list([0]) * (self.max_len - len(cancion))  # Padding
        else:
          cancion = cancion[:self.max_len]

        song_tensor = torch.tensor(cancion)

        return song_tensor,self.labels[idx]

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader

# Configuración del dispositivo (GPU si está disponible, de lo contrario CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paso el dataset a tokens para pasarlo al modelo
X_train_tokenizado = []
for cancion in X_train:
    cancion_tokenizada = [corpus.index(palabra) for palabra in cancion.split()]
    X_train_tokenizado.append(cancion_tokenizada)

# Creo el dataset
song_dataset = SongDataset(X_train_tokenizado, y_train,100)

# Creamos el DataLoader para entrenar el modelo
train_loader = DataLoader(song_dataset, batch_size=16, shuffle=True)

# Entrenamiento

# Inicialización del modelo
embedding_dim = 100  # Dimensión de los embeddings
hidden_dim = 128  # Tamaño de la capa oculta
num_classes = 3  # TINI, CALLEJERO, SPINETTA
vocab_size = len(corpus)  # Tamaño del vocabulario

model = Clasificador(vocab_size, embedding_dim, hidden_dim, num_classes)

# Función de pérdida y optimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Número de épocas para entrenar
num_epochs = 10


# Entrenamiento
for epoch in range(num_epochs):
    model.train()  # Modo entrenamiento
    for texts, labels in train_loader:

        # Mover los datos a la GPU
        texts, labels = texts.to(device), labels.to(device)

        optimizer.zero_grad()  # Limpiar los gradientes

        outputs = model(texts)  # Pasada fordward

        loss = criterion(outputs, labels)  # Calcular la pérdida

        loss.backward()  # Pasada backward

        optimizer.step()  # Actualizar los parámetros

    # Imprimir el progreso
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}")

Epoch 1/10, Loss: 0.8868
Epoch 2/10, Loss: 0.9020
Epoch 3/10, Loss: 0.4469
Epoch 4/10, Loss: 0.7085
Epoch 5/10, Loss: 0.5960
Epoch 6/10, Loss: 0.5794
Epoch 7/10, Loss: 0.5183
Epoch 8/10, Loss: 0.5925
Epoch 9/10, Loss: 0.3527
Epoch 10/10, Loss: 0.3774


In [ ]:
# Paso el dataset a tokens para pasarlo al modelo
X_test_tokenizado = []
for cancion in X_test:
    cancion_tokenizada = [corpus.index(palabra) for palabra in cancion.split()]
    X_test_tokenizado.append(cancion_tokenizada)

# Crear el dataset de prueba
test_dataset = SongDataset(X_test_tokenizado, y_test, 100)

# Crear un DataLoader para cargar los datos de prueba
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

# Función para evaluar el modelo
def evaluate_model(model, test_loader):
    model.eval()  # Poner el modelo en modo evaluación
    correct = 0
    total = 0

    with torch.no_grad():  # Desactivar el cálculo de gradientes
        for data, labels in test_loader:
            outputs = model(data)  # Obtener las predicciones del modelo
            _, predicted = torch.max(outputs, 1)  # Obtener la clase con mayor probabilidad
            total += labels.size(0)  # Número total de muestras
            correct += (predicted == labels).sum().item()  # Contar las predicciones correctas

    accuracy = 100 * correct / total  # Calcular la precisión en porcentaje
    return accuracy

# Evaluar el modelo
accuracy = evaluate_model(model, test_loader)

# Imprimir la precisión
print(f"Precisión en el conjunto de prueba: {accuracy:.2f}%")

Precisión en el conjunto de prueba: 21.43%


In [ ]:
def prediccion(cancion):
  cancion_tokenizada = [corpus.index(palabra) for palabra in cancion.split()]
  cancion_tokenizada = torch.tensor(cancion_tokenizada)
  cancion_tokenizada = cancion_tokenizada.unsqueeze(0)
  output = model(cancion_tokenizada)
  _, predicted = torch.max(output, 1)
  return categorias[predicted.item()]


El problema que tiene el modelo entrenado es que estamos entrenando nuestra propia capa de embeddings. En lugar de entrenar la capa de embeddings, es preferible usar algo preentrenado

En español: https://github.com/dccuchile/spanish-word-embeddings?tab=readme-ov-file#glove-embeddings-from-sbwc

## Usando embeddings pre entrenados

In [5]:
# Cargamos embeddings de GLOVE
embedding_file = 'glove.6B.100d.txt'
ruta_embedding = f'/content/drive/MyDrive/proyectos/NLP'
ruta_carpeta = os.listdir(ruta_embedding)
ruta_archivo = os.path.join(ruta_embedding, embedding_file)
embeddings_index = {}
with open(ruta_archivo, 'r') as archivo_txt:
  for line in archivo_txt:
    values = line.split()
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = vector

# Mostrar el vector para una palabra de ejemplo
print(embeddings_index['king'])

[-0.32307  -0.87616   0.21977   0.25268   0.22976   0.7388   -0.37954
 -0.35307  -0.84369  -1.1113   -0.30266   0.33178  -0.25113   0.30448
 -0.077491 -0.89815   0.092496 -1.1407   -0.58324   0.66869  -0.23122
 -0.95855   0.28262  -0.078848  0.75315   0.26584   0.3422   -0.33949
  0.95608   0.065641  0.45747   0.39835   0.57965   0.39267  -0.21851
  0.58795  -0.55999   0.63368  -0.043983 -0.68731  -0.37841   0.38026
  0.61641  -0.88269  -0.12346  -0.37928  -0.38318   0.23868   0.6685
 -0.43321  -0.11065   0.081723  1.1569    0.78958  -0.21223  -2.3211
 -0.67806   0.44561   0.65707   0.1045    0.46217   0.19912   0.25802
  0.057194  0.53443  -0.43133  -0.34311   0.59789  -0.58417   0.068995
  0.23944  -0.85181   0.30379  -0.34177  -0.25746  -0.031101 -0.16285
  0.45169  -0.91627   0.64521   0.73281  -0.22752   0.30226   0.044801
 -0.83741   0.55006  -0.52506  -1.7357    0.4751   -0.70487   0.056939
 -0.7132    0.089623  0.41394  -1.3363   -0.61915  -0.33089  -0.52881
  0.16483  -0.98878

In [15]:
# Pruebas BERT
emb = embeddings_index['king'] - embeddings_index['man']
print(embeddings_index.keys()[list(embeddings_index.values()).index(emb)])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [21]:
import gzip

# Descarga
!wget "http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2"

# Abro el archivo
# open('glove-sbwc.i25.vec', 'wb').write(gzip.open('glove-sbwc.i25.vec.gz', 'rb').read())

--2024-11-21 18:33:46--  http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2
Resolving cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)... 200.16.17.55
Connecting to cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)|200.16.17.55|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2 [following]
--2024-11-21 18:33:47--  https://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2
Connecting to cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)|200.16.17.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 818175453 (780M) [application/x-bzip2]
Saving to: ‘SBW-vectors-300-min5.txt.bz2’

                SBW   0%[                    ]   4.68M   107KB/s    eta 2h 7m  ^C


In [19]:
!tar -xzvf file.tar.gz

2447751199

In [27]:
!pip install gensim.models.wrappers

ERROR: Could not find a version that satisfies the requirement gensim.models.wrappers (from versions: none)
ERROR: No matching distribution found for gensim.models.wrappers


In [30]:
!pip install --upgrade gensim

In [31]:
# opción 1: cargar todos los vectores desde el formato binario (lento, requiere mucha memoria)
from gensim.models.wrappers import FastText
wordvectors_file = 'fasttext-sbwc.3.6.e20'
wordvectors = FastText.load_fasttext_format(wordvectors_file)

ModuleNotFoundError: No module named 'gensim.models.wrappers'